## Steps:


1. Set visualization folder and `mesh_args` and initialize `setup=ContactMechanicsBiotISC(mesh_args, folder_name, **kwargs)`
    - Runs `super().__init(params)` (`ContactMechanicsBiot`)
        - Set default `time`, `time_step`, `end_time` (Overwritten below)
        - Set default `scalar_scale`, `length_scale` **(TO BE OVERWRITTEN)**
        - Runs `super().__init(params)` (`ContactMechanics`)
            - Sets `params`
            - Sets `viz_folder_name` as value of `params['folder_name']`
                - This is decided at top level with argument `folder_name`
    - Set `file_name`
        - Solution file root name
    - Set `_set_time_parameters()`
        - `time` (current time)
        - `time_step`
        - `end_time`
        - `initial_time_step` (starting `time`)
    - **TODO**: Overwrite `scalar_scale` and `length_scale`
    - Set `source_scalar_borehole_shearzone`
        - Name of shearzone and borehole to use as injection source.
    - Set `shearzone_names`
        - Which shearzones to include as fractures
    - Set `mesh_args`
    - Set `box`
        - Bounding box of domain
    - Set `isc`
        - ISC Dataset.
        - Gets `kwargs['data_path']` if available (to find data source)
    - Run `well_cells()` (in `ContactMechanicsBiotISC`)
        - Tag the injection source cell and store in grid data.
            - `g.tags['well_cells'] = tags`
                - 1 for injection, -1 for production, 0 for nothing.
            - `pp.set_state(d, {"well": tags.copy()})`

2. Set parameters for Newton solver.

3. `pp.run_time_dependent_model(setup, params)`

### Now, we need to ensure all parameters are set as expected, and all methods do the things they are supposed to.

Inside `pp.run_time_dependent_model()`:

> `setup.prepare_simulation()` (in `ContactMechanicsBiotISC`)

- Run `create_grid()` (In `pp.ContactMechanicsBiot`)
    - Use `shearzone_names` and `box` to create fracture network 
    - Use `mesh_args` to mesh and `viz_folder_name` to store output file.
    - set `self.gb`, `self.Nd`
    - Add `name` to `GridBucket` data on each grid.
        - Tag the fracture grids with `S1_1`, `S1_2` etc.


- Run `set_parameters()` (in `pp.ContactMechanicsBiot`)
    - Run `set_scalar_parameters()` (In `pp.ContactMechanicsBiot`)
        * bc set with 
            - Run `bc_type_scalar(g)` (in `ContactMechanicsBiotISC`)
                - CURRENT: Dirichlet type bottom, neumann otherwise
                - **TODO**: MAKE HYDROSTATIC BC
            - Run `bc_values_svalar(g) (in `ContactMechanicsBiotISC`)
                - CURRENT: 1 on top, 0 otherwise
                - **TODO**: See right above - hydrostatic. 
        * `source_scalar` (in `ContactMechanicsBiotISC`)
            - set flow_rate:
            - Run `source_flow_rate` (in `ContactMechanicsBiotISC`)
                - 3 liters
            - Set source to 3 liters in `g.tags['well_cells']`
                - Multiplied by time_step
                    - **TODO**: Ask if this is correct.
        * Aperture computed
            - `compute_aperture(g)` (in `pp.ContactMechanicsBiot`)
                - **TODO**: OVERWRITE TO INTERPOLATION IN FRACTURES
                    - See existing method in `porepy paper`
        * `biot_alpha(g)` (in `pp.ContactMechanicsBiot`)
            - Defaults to 1.
        * non-method assignments:
            - `tensor_scale`
            - `kappa` = `1 * tensor_scale`
                - **TODO**: See if this needs changing
                - _Note_: This is used on mortar grid too.
            - `mass_weight`
            - `specific_volume`
            - `diffusivity = kappa * specific_volume` (permeability)
                - `specific_volume = a` in fractures and `a^2` in fracture intersections.
                - **TODO**: Verify that aperture scaling is correct (see above TODO)
        * **Summary:** *scalar parameters set*:
            - `bc`
            - `bc_values`
            - `mass_weight = mass_weight * specific_volume`
            - `biot_alpha`
            - `source`
            - `second_order_tensor`
            - `time_step`: Set by `self.time_step`.
        * _On Mortar grid_:
            - Assign diffusivity in normal direction on fractures.
            - Run `compute_aperture`
            - Set `normal_diffusivity` on mortar grid.
    - Run `set_mechanics_parameters()` (in `ContactMechanicsBiotISC`)
        - Rock parameters:
        - **ON MATRIX GRID**:
            - Run `set_lam(g)` 
                - sets lambda Lamé parameter to 1.
                - **TODO**: Change this to custom lam. See documentation in method
            - Run `set_mu(g)`
                - sets mu Lamé parameter to 1.
                - **TODO**: Change to custom lam. see docs in method.
            - Set `FourthOrderTensor` from mu and lam.
            - Run `bc_type_mechanics(g)` (in `ContactMechanicsBiotISC` -> calls super() )
                - **TODO**: Custom mechanics BC type
            - Run `bc_values_mechanics(g)` (in `ContactMechanicsBiotISC` -> calls super() )
                - **TODO**: Custom mechanics BC values
            - Run `source_mechanics(g)` (in `pp.ContactMechanicsBiot` -> calls super() )
                - Set to zero (in `pp.ContactMechanics`)
            - **Summary:** _Mechanics parameters set*:
                - `bc`
                - `bc_values`
                - `source`
                - `fourth_order_tensor`
                - `time_step`: Set by `self.time_step`
                - `biot_alpha`
                    - Run `biot_alpha(g)` (in `pp.ContactMechanicsBiot`)
                        - Defaults to 1.
        - **ON FRACTURE GRID**:
            - Run `_set_friction_coefficient(g)` (in `pp.ContactMechanics`)
                - Defaults to 1
                - **TODO**: Should this be changed?
            - **Summary:** _Mechanics parameters set*:
                - `friction_coefficient`
                - `time_step`: Set by `self.time_step`
        * _On Mortar grid_:
            - Nothing is set here.


- Run `assign_variables()` (in `pp.ContactMechanicsBiot`)
    - Assign primary variables to the nodes and edges of the grid bucket.


- Run `assign_discretizations()` (in `pp.ContactMechanicsBiot`)
    - Assign discretizations to the nodes and edges of the grid bucket. 


- Run `initial_condition()` (in `pp.ContactMechanicsBiot`)
    - All initial conditions are saved to appropriate `data[pp.STATE]`
    - Initial guess for Newton iteration, scalar variable and bc_values (for time
        discretization).
    - Run `super().initial_condition()` (in `pp.ContactMechanics`)
        - Set displacements to zero on Nd-domain and fracture interfaces.
            - Thus displacement jump is zero.
        - Set tangential contact pressure to zero
        - Set normal contact pressure to -1
            - Thus, fractures are in contact.
    - Set scalar variable to zero.


- Run `discretize()` (in `pp.ContactMechanicsBiot`)
    - _Discretize all terms_
    - Set `assembler` if not already set.
    - Run `discretize_biot()` (in `pp.ContactMechanicsBiot`)
        - Discretizes the Nd-grid once
            - I.e. contact_mechanics is not discretized here.
    - Discretize source term on Nd-grid 
        - Since it is not covered by the Biot discretization.
    - Discretize lower-dimensional grids
        - This is done with default method, since there is no Biot discretization here.


- Run `initialize_linear_solver()` (in `pp.ContactMechanicsBiot`)

    - Set solver with `self.params.get("linear_solver", "direct")`
        - Default is `direct`.


- Run `set_viz()` (in `ContactMechanicsBiotISC`)
    - Set `viz`
        - The `pp.Exporter` class
    - Set `export_times`
        - Used by `export_pvd`
    - Set `u_exp` and `p_exp`
        - Name of variables to export
        - Note - These are different from the normal variable names because they need to be manipulated before exporting. The manipulation is done in `export_step()`
    - Set `export_fields`
        - List of variables to export in `export_step()`

> Recall: We are in `pp.run_time_dependent_model(setup, params)`

- Run `setup._is_nonlinear_problem()` (in `ContactMechanicsBiotISC`)
    - Returns `True`
- Because above is `True`:
    - `solver = pp.NewtonSolver(params)`
        - Recall `params` is input to `pp.run_time_dependent_model()`
- **Start time loop**:
- `while setup.time < t_end:`
    - `setup.time += setup.time_step`
    - Run `solver.solve(setup)` (in `pp.NewtonSolver`)
        - Run `setup.before_newton_loop()` (in `ContactMechanicsBiotISC`)
            - Run `set_parameters()` (see above)
                - Updates all parameters
            - Run `discretize()` (see above)
                - **TODO**: See which terms you don't need to discretize
                    - I.e.: Avoid discretizing Biot on Nd-grid.
        - Run `prev_sol = setup.get_state_vector()` (in `pp.ContactMechanics`)
            - Gets previous solution for all states from `data[pp.STATE][var]`
        - **Go in to Newton iteration**
            - _Check if max iterations not exceeded, or convergence is reached._
            - Run `setup.before_newton_iteration()` (see above)
                - **TODO**: Re-discretize non-linear term
            - Set `lin_tol = np.minimum(1e-4, error_norm)`
                - `error_norm = 1` by default (hard coded)
            - Run `sol = self.iteration(setup, lin_tol)` (in `pp.NewtonSolver`)
                - Run `assemble_and_solve_linear_system(lin_tol)` (in `pp.ContactMechanicsBiot`)
                    - Solves the problem determined with method determined by `self.linear_solver`
            - Run `setup.after_newton_iteration(sol)` (in `pp.ContactMechanicsBiot`)
                - Run `update_state(sol)` (in `pp.ContactMechanics`)
            - Run `check_convergence(sol, prev_sol, self.params)` (in `pp.ContactMechanics`)
                - Calculate relative error in displacement on the Nd grid
                - Return `error_norm, is_converged, is_diverged`
            - First, if `is_diverged`
                - `raise ValueError`. Newton method diverged.
            - Then, if `is_converged`:
                - Run `after_newton_convergence(sol, errors, iteration_counter)` (in `pp.ContactMechanicsBiot`)
                    - Distribute solution to `data[pp.STATE]`
                    - Run `save_mechanical_bc_values()` (in `pp.ContactMechanicsBiot`)
                        - Update mechanical bc values if they are time-dependent.
                    - **TODO**: Do something with `errors` and `iteration_counter`?
        - **Newton iteration done**:
        - Check, if not `is_converged`:
            - Run `after_newton_failure()`
                - `raise ValueError`
    - **One time step is done**: Return to loop in `pp.run_time_dependent_model(...)`
- Done!

* When we are done, remember to export solution.